In [1]:
import numpy as np
import matplotlib.pyplot as plt

import xtrack as xt
import xpart as xp
import xobjects as xo
import xcoll as xc

new_builder test to put in collimators as pipes

In [154]:
#Line with just elements
env = xt.load(xc._pkg_root.parent / 'examples' / 'machines' / 'sps_q20_inj.json')
tt = env.sps.get_table()
tt = tt.rows[[not nn.startswith('drift_') for nn in tt.name]]
tt = tt.rows[[not 'Drift' in ee and not 'Marker' in ee for  ee in  tt.element_type]]
length_builder = tt.s[-1]
assert tt.name[-1] == '_end_point'
tt = tt.rows[:-1]

Loading line from dict:   0%|          | 0/4037 [00:00<?, ?it/s]

In [142]:
builder = env.new_builder(length=length_builder)
builder.particle_ref = env.sps.particle_ref

for nn in tt.name:
    builder.place(nn, at=tt['s_start', nn], anchor='start')

In [99]:
# line2 = builder.build()
# line2.particle_ref = env.sps.particle_ref
# tt_line2 = line2.get_table()

In [127]:
line_aper = xt.load(xc._pkg_root.parent / 'examples' / 'machines' / 'sps_with_aperture_inj_q20.json')
env_aper = line_aper.env
env_aper['sps'] = line_aper
tt_aper = env_aper.sps.get_table()
tt_aper = tt_aper.rows[[et.startswith('Limit') for et in tt_aper.element_type]]

Loading line from dict:   0%|          | 0/17781 [00:00<?, ?it/s]

Done loading line from dict.           


In [128]:
element_numbers = [int(nn.split('.')[1]) for nn in tt.name]

In [115]:
for nn in tt_aper.name:
    env.elements[nn] = env_aper.elements[nn].copy()
    spl = nn.split('.')
    ele_number = int(spl[1])
    ele_base = spl[0]
    aper_letter = spl[2].split('_')[0]

    if 'bp' in ele_base:
        continue
    elif ele_number in element_numbers and not ele_base.startswith('v') and not ele_base.startswith('act'):
        if aper_letter == 'a':
            builder.place(nn, at=0, from_=f'{ele_base}.{ele_number}@start', anchor='end')
        elif aper_letter == 'b':
            builder.place(nn, at=0, from_=f'{ele_base}.{ele_number}@end', anchor='start')
        else:
            builder.place(nn, at=tt_aper.rows[nn].s_start[0])#, from_=nn+'@start', anchor='start')
    else:
        builder.place(nn, at=tt_aper.rows[nn].s_start[0])#, from_=nn+'@start', anchor='start')

In [116]:
line2 = builder.build()
line2.particle_ref = env.sps.particle_ref
tt_line2 = line2.get_table()

AssertionError: Negative drift length: -0.002189373269089856, upstream of lse.10602

In [63]:
tt_aper.rows['.*10107.*']

Table: 2 rows, 11 cols
name                         s element_type isthick isreplica parent_name iscollective ...
mdv.10107.a_aper       31.3633 LimitEllipse   False     False None               False
mdv.10107.b_aper       31.6121 LimitEllipse   False     False None               False

In [79]:
tt.rows['.*10107.*']

Table: 1 row, 11 cols
name                  s element_type isthick isreplica parent_name iscollective       s_start ...
mdv.10107       31.3629 Multipole       True     False None               False       31.3629

In [104]:
from pathlib import Path

In [105]:
line_old = xt.Line.from_json(Path('/Users/lisepauwels/sps_simulations/injection_lines/sps_with_aperture_inj_q20_beam_sagitta4.json'))

Loading line from dict:   0%|          | 0/36381 [00:00<?, ?it/s]

Done loading line from dict.           


In [106]:
tt_old = line_old.get_table()

In [117]:
tt_old.rows['.*10602.*']

Table: 10 rows, 11 cols
name                             s element_type            isthick isreplica parent_name ...
vttb.10602.a_aper           189.84 LimitEllipse              False     False None       
lse.10602_entry            189.943 Marker                    False     False None       
lse.10602..entry_map       189.943 ThinSliceSextupoleEntry   False     False lse.10602  
lse.10602..0               189.943 ThickSliceSextupole        True     False lse.10602  
vttb.10602.z_aper          189.947 LimitEllipse              False     False None       
vttb.10602.b_aper          189.947 LimitEllipse              False     False None       
lse.10602..1               189.947 ThickSliceSextupole        True     False lse.10602  
lse.10602..exit_map        190.683 ThinSliceSextupoleExit    False     False lse.10602  
lse.10602_exit             190.683 Marker                    False     False None       
vttb.10602.c_aper          191.261 LimitEllipse              False     False None 

Check number of apertures per element, remove the ones that do not need to be there and fix names

In [129]:
pipe_names = {}
for nn in tt_aper.name:
    name_pipe = nn.split('.')[0] + '.' + nn.split('.')[1]
    if name_pipe in pipe_names:
        pipe_names[name_pipe] += 1
    else:
        pipe_names[name_pipe] = 1

In [130]:
for nn in pipe_names:
    if pipe_names[nn] > 2:
        print(nn, pipe_names[nn])

vttc.10203 4
vttv.10205 4
vttb.10402 4
vttb.10602 4
vttc.10802 4
vttv.10805 4
vttd.11001 4
vttc.11201 4
vttv.11205 4
vttb.11402 4
vedu.11432 3
vttb.11601 4
vcsmc.11802 4
vknv.11839 3
vcsmb.11897 6
vknl.11974 3
vttl.12002 3
vttm.12202 3
vttb.12402 4
vttc.12602 4
vttv.12605 4
vttc.12802 4
vttc.13002 4
vttc.13201 4
vttv.13205 4
vttd.13401 4
vttb.13602 4
vttv.20205 4
vttc.20401 4
vttb.20602 4
vttc.20801 4
vttv.20805 4
vttb.21202 4
vttv.21405 4
vewj.21753 3
vmek.21798 3
vmem.21802 3
veco.21877 3
vecl.21878 3
vttl.22002 3
vttm.22202 3
vttb.22402 4
vttc.22602 4
vttv.22605 4
vttc.22802 4
vttc.23002 4
vttc.23201 4
vttv.23205 4
vttd.23401 4
vttc.23602 4
vttv.23605 4
vttb.30202 4
vttd.30401 4
vttb.30602 4
vttc.30802 4
vttv.30805 4
vttd.31001 4
vttv.31205 4
vttb.31402 4
vttz.31601 3
vttl.32002 3
vttm.32202 3
vttb.32402 4
vttc.32602 4
vttv.32605 4
vttc.32802 4
vttb.33002 4
vttc.33201 4
vttv.33205 4
vttb.33602 4
vttc.40201 4
vttv.40205 4
vttb.40402 4
vttb.40602 4
vttc.40802 4
vttv.40805 4
vttd.41001

In [144]:
for nn in pipe_names:
    base = nn.split('.')[0] + '.' + nn.split('.')[1]

    if 'v' in nn:
        if pipe_names[nn] > 2:
            apers = tt_aper.rows[nn + '.*'].name
            env.elements[f'{nn}.a_aper'] = env_aper.elements[apers[0]].copy()
            env.elements[f'{nn}.b_aper'] = env_aper.elements[apers[-1]].copy()
            builder.place(f'{nn}.a_aper', at=tt_aper.rows[apers[0]].s[0])
            builder.place(f'{nn}.b_aper', at=tt_aper.rows[apers[-1]].s[0])
        else:
            if nn != 'veqf.10010':
                env.elements[f'{nn}.a_aper'] = env_aper.elements[f'{nn}.a_aper'].copy()
                builder.place(f'{nn}.a_aper', at=tt_aper.rows[f'{nn}.a_aper'].s[0])
            env.elements[f'{nn}.b_aper'] = env_aper.elements[f'{nn}.b_aper'].copy()
            builder.place(f'{nn}.b_aper', at=tt_aper.rows[f'{nn}.b_aper'].s[0])
        

    else:
        if base in tt.name:
            if pipe_names[nn] > 2:
                apers = tt_aper.rows[nn + '.*'].name
                env.elements[f'{nn}.a_aper'] = env_aper.elements[apers[0]].copy()
                env.elements[f'{nn}.b_aper'] = env_aper.elements[apers[-1]].copy()
                builder.place(f'{nn}.a_aper', at=0, from_=f'{nn}@start', anchor='end')
                builder.place(f'{nn}.b_aper', at=0, from_=f'{nn}@end', anchor='start')
            else:
                env.elements[f'{nn}.a_aper'] = env_aper.elements[f'{nn}.a_aper'].copy()
                env.elements[f'{nn}.b_aper'] = env_aper.elements[f'{nn}.b_aper'].copy()
                builder.place(f'{nn}.a_aper', at=0, from_=f'{nn}@start', anchor='end')
                builder.place(f'{nn}.b_aper', at=0, from_=f'{nn}@end', anchor='start')
    # if base in tt.name:
    #     if pipe_names[nn] > 2:
    #         apers = tt_aper.rows[nn + '.*'].name
    #         env.elements[f'{nn}.a_aper'] = env_aper.elements[apers[0]].copy()
    #         env.elements[f'{nn}.b_aper'] = env_aper.elements[apers[-1]].copy()

    #         if 'v' in nn:
    #             builder.place(f'{nn}.a_aper', at=tt_aper.rows[f'{nn}.a_aper'].s[0])
    #             builder.place(f'{nn}.b_aper', at=tt_aper.rows[f'{nn}.b_aper'].s[0])

    #         else:
    #             builder.place(f'{nn}.a_aper', at=0, from_=f'{nn}@start', anchor='end')
    #             builder.place(f'{nn}.b_aper', at=0, from_=f'{nn}@end', anchor='start')
    #     else:
    #         if nn in tt_aper.name:
    #             env.elements[f'{nn}.a_aper'] = env_aper.elements[f'{nn}.a_aper'].copy()
    #             env.elements[f'{nn}.b_aper'] = env_aper.elements[f'{nn}.b_aper'].copy()
    #             if 'v' in nn:
    #                 builder.place(f'{nn}.a_aper', at=tt_aper.rows[f'{nn}.a_aper'].s[0])
    #                 builder.place(f'{nn}.b_aper', at=tt_aper.rows[f'{nn}.b_aper'].s[0])

    #             else:
    #                 builder.place(f'{nn}.a_aper', at=0, from_=f'{nn}@start', anchor='end')
    #                 builder.place(f'{nn}.b_aper', at=0, from_=f'{nn}@end', anchor='start')
        

KeyError: 'vtaf.51632.b_aper'

In [133]:
tt_aper.rows['teca.*']

Table: 1 row, 11 cols
name                          s element_type isthick isreplica parent_name iscollective ...
teca.41777.a_aper       4020.64 LimitEllipse   False     False None               False

In [139]:
line2 = builder.build()
line2.particle_ref = env.sps.particle_ref
tt_line2 = line2.get_table()

In [140]:
tt_line2

Table: 2977 rows, 11 cols
name                   s element_type isthick isreplica parent_name iscollective       s_start ...
qf.10010               0 Quadrupole      True     False None               False             0
drift_1909         3.085 Drift           True     False None               False         3.085
mba.10030        3.44501 RBend           True     False None               False       3.44501
drift_1910       9.70503 Drift           True     False None               False       9.70503
mba.10050         10.105 RBend           True     False None               False        10.105
drift_1911       16.3651 Drift           True     False None               False       16.3651
mbb.10070        16.7551 RBend           True     False None               False       16.7551
drift_1912       23.0151 Drift           True     False None               False       23.0151
mbb.10090        23.3951 RBend           True     False None               False       23.3951
drift_1913       29.

In [161]:
tt.rows['qd.31110<<1':'qd.3110>>1']

KeyError: "Cannot find 'qd.3110' in column 'name'"

In [168]:
tt.rows['qd.31110<<10':'qd.31110>>10']

Table: 21 rows, 11 cols
name                  s element_type isthick isreplica parent_name iscollective       s_start ...
mba.30970       2608.55 RBend           True     False None               False       2608.55
mba.30990       2615.21 RBend           True     False None               False       2615.21
mdh.31007       2623.18 Multipole       True     False None               False       2623.18
qf.31010        2623.82 Quadrupole      True     False None               False       2623.82
mba.31030       2627.27 RBend           True     False None               False       2627.27
mba.31050       2633.93 RBend           True     False None               False       2633.93
mbb.31070       2640.58 RBend           True     False None               False       2640.58
mbb.31090       2647.22 RBend           True     False None               False       2647.22
lsd.31105       2654.61 Sextupole       True     False None               False       2654.61
mdv.31107       2655.19 Multipol

In [170]:
tt.rows['qf.40010<<10':'qf.40010>>10']

Table: 21 rows, 11 cols
name                  s element_type isthick isreplica parent_name iscollective       s_start ...
lsd.33505       3422.56 Sextupole       True     False None               False       3422.56
mdv.33507       3423.13 Multipole       True     False None               False       3423.13
qd.33510        3423.77 Quadrupole      True     False None               False       3423.77
mbb.33530        3427.2 RBend           True     False None               False        3427.2
mbb.33550       3433.84 RBend           True     False None               False       3433.84
mba.33570       3440.49 RBend           True     False None               False       3440.49
mba.33590       3447.15 RBend           True     False None               False       3447.15
loe.33602       3453.73 Octupole        True     False None               False       3453.73
lsf.33605       3454.55 Sextupole       True     False None               False       3454.55
mdh.33607       3455.12 Multipol

In [173]:
tt.rows['qd.10110<<10':'qd.10110>>10']

Table: 0 rows, 11 cols

In [187]:
tt.rows['qf.10010<<10':]

Table: 10 rows, 11 cols
name                  s element_type isthick isreplica parent_name iscollective       s_start ...
lsd.63505       6878.32 Sextupole       True     False None               False       6878.32
mdv.63507        6878.9 Multipole       True     False None               False        6878.9
qd.63510        6879.53 Quadrupole      True     False None               False       6879.53
mbb.63530       6882.97 RBend           True     False None               False       6882.97
mbb.63550       6889.61 RBend           True     False None               False       6889.61
mba.63570       6896.26 RBend           True     False None               False       6896.26
mba.63590       6902.92 RBend           True     False None               False       6902.92
loe.63602        6909.5 Octupole        True     False None               False        6909.5
lsf.63605       6910.32 Sextupole       True     False None               False       6910.32
mdh.63607       6910.89 Multipol

In [186]:
tt.rows[:'qd.10110>>10']

Table: 18 rows, 11 cols
name                  s element_type isthick isreplica parent_name iscollective       s_start ...
qf.10010              0 Quadrupole      True     False None               False             0
mba.10030       3.44501 RBend           True     False None               False       3.44501
mba.10050        10.105 RBend           True     False None               False        10.105
mbb.10070       16.7551 RBend           True     False None               False       16.7551
mbb.10090       23.3951 RBend           True     False None               False       23.3951
lsd.10105       30.7879 Sextupole       True     False None               False       30.7879
mdv.10107       31.3629 Multipole       True     False None               False       31.3629
qd.10110        31.9979 Quadrupole      True     False None               False       31.9979
mbb.10130       35.4329 RBend           True     False None               False       35.4329
mbb.10150       42.0729 RBend   

In [ ]:
line_q20 = xt.Line.from_json(xc._pkg_root.parent / 'examples' / 'machines' / 'sps_q20_inj.json')
line_q26 = xt.Line.from_json(xc._pkg_root.parent / 'examples' / 'machines' / 'sps_q26_inj.json')